<a href="https://colab.research.google.com/github/aidyai/-./blob/main/HTML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install einops flash_attn
!pip install autoawq

In [2]:
import torch

from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor
from transformers import AwqConfig
from transformers.image_utils import to_numpy_array, PILImageResampling, ChannelDimension
from transformers.image_transforms import resize, to_channel_dimension_format

from transformers import BitsAndBytesConfig, pipeline


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
#DEVICE = torch.device("cuda")
# Check if a GPU is available and set the device accordingly
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PROCESSOR = AutoProcessor.from_pretrained(
    "HuggingFaceM4/VLM_WebSight_finetuned",
    token="/root/.cache/huggingface/token",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!pip install bitsandbytes
!pip install accelerate

In [4]:
model_id = "HuggingFaceM4/VLM_WebSight_finetuned"

In [16]:
from transformers import BitsAndBytesConfig

double_quant_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_use_double_quant=True,
)

# load model in 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)



In [ ]:
model_double_quant = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

The repository for HuggingFaceM4/VLM_WebSight_finetuned contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/HuggingFaceM4/VLM_WebSight_finetuned.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
#MODEL = AutoModelForCausalLM.from_pretrained(
# attn_implementation="flash_attention_2",

#    token="/root/.cache/huggingface/token",
#    trust_remote_code=True,
#    torch_dtype=torch.bfloat16,
#).to(DEVICE)

In [9]:
image_seq_len = model_double_quant.config.perceiver_config.resampler_n_latents
BOS_TOKEN = PROCESSOR.tokenizer.bos_token
BAD_WORDS_IDS = PROCESSOR.tokenizer(["<image>", "<fake_token_around_image>"], add_special_tokens=False).input_ids



In [10]:
def convert_to_rgb(image):
    # `image.convert("RGB")` would only work for .jpg images, as it creates a wrong background
    # for transparent images. The call to `alpha_composite` handles this case
    if image.mode == "RGB":
        return image

    image_rgba = image.convert("RGBA")
    background = Image.new("RGBA", image_rgba.size, (255, 255, 255))
    alpha_composite = Image.alpha_composite(background, image_rgba)
    alpha_composite = alpha_composite.convert("RGB")
    return alpha_composite

In [11]:
# The processor is the same as the Idefics processor except for the BILINEAR interpolation,
# so this is a hack in order to redefine ONLY the transform method
def custom_transform(x):
    x = convert_to_rgb(x)
    x = to_numpy_array(x)
    x = resize(x, (960, 960), resample=PILImageResampling.BILINEAR)
    x = PROCESSOR.image_processor.rescale(x, scale=1 / 255)
    x = PROCESSOR.image_processor.normalize(
        x,
        mean=PROCESSOR.image_processor.image_mean,
        std=PROCESSOR.image_processor.image_std
    )
    x = to_channel_dimension_format(x, ChannelDimension.FIRST)
    x = torch.tensor(x)
    return x

In [12]:
inputs = PROCESSOR.tokenizer(
    f"{BOS_TOKEN}<fake_token_around_image>{'<image>' * image_seq_len}<fake_token_around_image>",
    return_tensors="pt",
    add_special_tokens=False,
)

In [13]:
from PIL import Image
image = Image.open("/content/UI DESIGN.jpg")

In [14]:
#inputs["pixel_values"] = PROCESSOR.image_processor([image], transform=custom_transform)

In [15]:
inputs["pixel_values"] = PROCESSOR.image_processor([image], transform=custom_transform)
inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
generated_ids = model_double_quant.generate(**inputs, bad_words_ids=BAD_WORDS_IDS, max_length=4096)
generated_text = PROCESSOR.batch_decode(generated_ids, skip_special_tokens=True)[0]

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


RuntimeError: FlashAttention only supports Ampere GPUs or newer.

In [ ]:
print(generated_text)
